In [1]:
import pandas as pd
import numpy as np

import ast
import json

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
dfgenres= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\CSVNUEVOS\Genres.csv')
dfgenres.head()


,id,genres
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"
4,11862,"[{'id': 35, 'name': 'Comedy'}]"


In [4]:
dfgenres.shape

(44936, 2)

In [5]:
dfgenres.sort_values(by='id', ascending=False)

,id,genres
20051,469172,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'name': 'Drama'}]"
44871,468707,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
21732,468343,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"
44748,467731,"[{'id': 18, 'name': 'Drama'}]"
44559,465044,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'name': 'Drama'}]"
...,...,...
255,11,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 878, 'name': 'Science Fiction'}]"
473,6,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}, {'id': 80, 'name': 'Crime'}]"
17,5,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name': 'Comedy'}]"
12885,3,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}]"


In [6]:
dfgenres['genres'] = dfgenres['genres'].apply(ast.literal_eval) 

In [7]:
df_exploded = dfgenres.explode('genres', ignore_index=True) 
df_exploded.head()

,id,genres
0,862,"{'id': 16, 'name': 'Animation'}"
1,862,"{'id': 35, 'name': 'Comedy'}"
2,862,"{'id': 10751, 'name': 'Family'}"
3,8844,"{'id': 12, 'name': 'Adventure'}"
4,8844,"{'id': 14, 'name': 'Fantasy'}"


In [8]:
df_exploded['name_genres'] = df_exploded['genres'].apply(lambda x: x.get('name', '') if isinstance(x, dict) else '')
df_exploded['id_genres'] = df_exploded['genres'].apply(lambda x: x.get('id', '') if isinstance(x, dict) else '') 

In [9]:
df_exploded.head()

,id,genres,name_genres,id_genres
0,862,"{'id': 16, 'name': 'Animation'}",Animation,16
1,862,"{'id': 35, 'name': 'Comedy'}",Comedy,35
2,862,"{'id': 10751, 'name': 'Family'}",Family,10751
3,8844,"{'id': 12, 'name': 'Adventure'}",Adventure,12
4,8844,"{'id': 14, 'name': 'Fantasy'}",Fantasy,14


In [10]:
df_exploded.drop(columns='genres')

,id,name_genres,id_genres
0,862,Animation,16
1,862,Comedy,35
2,862,Family,10751
3,8844,Adventure,12
4,8844,Fantasy,14
...,...,...,...
92607,67758,Action,28
92608,67758,Drama,18
92609,67758,Thriller,53
92610,227506,,


In [11]:
grouped_df = df_exploded.groupby('id').agg({'name_genres': lambda x: "'" + "' '".join(x) + "'",
                                           'id_genres': lambda x: "'" + ','.join(map(str, x)) + "'"}).reset_index()

grouped_df 



,id,name_genres,id_genres
0,2,'Drama' 'Crime',"'18,80'"
1,3,'Drama' 'Comedy',"'18,35'"
2,5,'Crime' 'Comedy',"'80,35'"
3,6,'Action' 'Thriller' 'Crime',"'28,53,80'"
4,11,'Adventure' 'Action' 'Science Fiction',"'12,28,878'"
...,...,...,...
44902,465044,'Fantasy' 'Drama',"'14,18'"
44903,467731,'Drama','18'
44904,468343,'Drama' 'Romance',"'18,10749'"
44905,468707,'Romance' 'Comedy',"'10749,35'"


In [12]:
grouped_df.sort_values(by='id', ascending=False)

,id,name_genres,id_genres
44906,469172,'Fantasy' 'Drama',"'14,18'"
44905,468707,'Romance' 'Comedy',"'10749,35'"
44904,468343,'Drama' 'Romance',"'18,10749'"
44903,467731,'Drama','18'
44902,465044,'Fantasy' 'Drama',"'14,18'"
...,...,...,...
4,11,'Adventure' 'Action' 'Science Fiction',"'12,28,878'"
3,6,'Action' 'Thriller' 'Crime',"'28,53,80'"
2,5,'Crime' 'Comedy',"'80,35'"
1,3,'Drama' 'Comedy',"'18,35'"


In [13]:
hola2= grouped_df[grouped_df['id']== '862']
hola2

,id,name_genres,id_genres


In [14]:
grouped_df['id'].unique()

array([     2,      3,      5, ..., 468343, 468707, 469172], dtype=int64)

In [15]:
df_exploded['id'].unique()

array([   862,   8844,  15602, ...,  67758, 227506, 461257], dtype=int64)

In [16]:
## creo el CSV DE LOS GENEROS CON LOS IDS DE LAS PELICULAS
grouped_df.to_csv('GenerosFinalPORid.csv', index=False)